### IEMS 308 - Data Science, Text Analytics Project

1. The data can be found at the following [link](www.google.com).
2. Build a text pre-processing engine
3. Build a simple AI computation engine.

#### Required Tasks
1. Extract all company names from the files.
2. Extract all numbers involving percentages. Note that sometimes the corpus has “0.5%” and other times “point five percent.” (and there might be other forms)
3. Extract all names of CEO’s.

**As deliverables you have to provide by Monday, March 2 at 5 pm on github:**
1. Files with extracted entities (3 files)
2. A document specifying a list of the features used, the process taken (regex’s used, other preprocessing techniques used, etc), the selected classification model, and the performance of your model.
3. The source code behind your NER classifier.

#### Documentation:

1. File conversion not required, all files already in txt format with yyyy-mm-dd format.
2. Import files with `osdir` into dictionary with date as `key`.




In [1]:
import os
from tqdm import tqdm

directory = r'/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/data' #directory to search for data
filenames = [] #new list to hold filenames
dates = [] #new list to hold date keys

#traverse directory and import all relevant data
for filename in os.listdir(directory):
    if filename.endswith(".txt"): #just double check correct filetype
        url = os.path.join(directory,filename)
        dates.append(filename.strip('.txt'))
        filenames.append(url)
    else: continue        

In [2]:
filenames.sort() #organise into date order
filenames[:3]

['/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/data/2013-01-01.txt',
 '/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/data/2013-01-02.txt',
 '/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/data/2013-01-03.txt']

In [3]:
%%time
import string
printable = set(string.printable)

content = []
for filename in filenames:
    with open(filename,encoding='utf8',errors='ignore') as f:
        document = f.read()
        document = ''.join(filter(lambda x: x in printable, document))
        document = document.replace('\n', ' ')
        content.append(document)
f.close()        

CPU times: user 9.03 s, sys: 339 ms, total: 9.37 s
Wall time: 9.59 s


In [4]:
# content[0]

In [5]:
import nltk, string, re

#### Progammatization
The first step to doing this is to write a function that performs the process on each document within tome.

In [ ]:
# keys = dates
# keys.sort()
# values = content
# holster = dict(zip(keys,values))

### Preprocess `.csv` data

Extract training set from `csv` files.

In [6]:
import pandas as pd

#extract all in one step

# ceo_labels = ['','2','full']
cents_labels = ['cent']
corps_labels = ['first']

training_ceo = pd.read_csv('/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/trained/ceo.csv',header=None,encoding='utf-8')
training_cents = pd.read_csv('/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/trained/percentage.csv',names=cents_labels,header=None,encoding='utf-8')
training_corp = pd.read_csv('/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/trained/companies.csv',header=None,names=corps_labels,encoding='utf-8')

In [7]:
training_ceo[2] = [' '.join(s.split()) for s in training_ceo[2]]
#convert all to list
train_ceo = list(set(training_ceo[2].tolist()))
train_corp = training_corp['first'].to_list()
train_cent = training_cents['cent'].to_list()

In [9]:
import spacy

nlp = spacy.load("en_core_web_sm", disable=['tagger','parser'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))
doc = nlp(content[0])
# for token in doc.ents:
#     print(token.text, token.start_char, token.end_char,token.label_)
#     print(token.text, token.pos_, token.dep_)

In [10]:
%time processed = [nlp(article) for article in tqdm(content[0:len(content)])]

100%|██████████| 730/730 [20:08<00:00,  1.66s/it]

CPU times: user 37min 31s, sys: 6min 57s, total: 44min 28s
Wall time: 20min 8s


In [14]:
import pickle
def pickle_me_hearties(pval,pstart,pend):
    '''
    pval is the file iterator
    pstart is start slice
    pend is end slice
    '''
    with open('processed'+str(pval)+'.pkl','wb') as pf:
        pickle.dump(processed[pstart:pend],pf,protocol=pickle.HIGHEST_PROTOCOL)
    

In [15]:
def return_me_hearties(plist):
    '''
    plist is the list of file iterators
    '''
    full_store=[]
    for i in tqdm(plist):
        with open('processed'+str(i)+'.pkl','rb') as pf:
            var = pickle.load(pf)
            full_store += var
    return full_store
    

In [13]:
# %time pickle_me_hearties(1,0,10)
# %time pickle_me_hearties(2,151,300)
# %time pickle_me_hearties(3,301,450)
# %time pickle_me_hearties(4,451,600)
# %time pickle_me_hearties(5,600,729)

In [ ]:
# %time new = return_me_hearties([1,2,3,4,5])

In [ ]:
# processed=new
# type(processed[0])

In [16]:
def matching_engine_3000(values,match):
    '''
    values is the nlp processed docs
    match is the entity type to match e.g. PERSON
    '''
    engine_results = [ent.text for ent in values.ents if ent.label_ == match]
    return engine_results

In [17]:
full_person_list = []
for i in tqdm(range(len(processed))):
    people = matching_engine_3000(processed[i],"PERSON")
    full_person_list.extend(people)


100%|██████████| 730/730 [00:05<00:00, 132.28it/s]


In [18]:
full_company_list = []
for i in tqdm(range(len(processed))):
    companies = matching_engine_3000(processed[i],"ORG")
    full_company_list.extend(companies)

100%|██████████| 730/730 [00:02<00:00, 248.97it/s]


In [ ]:
# full_company_list[1:100]

In [19]:
%%time
takehome_percentages =[]
for i in processed:
    takehome_percentages.extend(matching_engine_3000(i,"PERCENT"))

CPU times: user 2.15 s, sys: 22.8 ms, total: 2.17 s
Wall time: 2.19 s


In [20]:
takehome_percentages=list(set(takehome_percentages))
takehome_percent = pd.DataFrame(takehome_percentages)
takehome_percent.to_csv('takehome_percent.csv',index=False)

In [21]:
from spacy import displacy
displacy.render(processed[0],style='ent',jupyter=True)

In [25]:
def filter_sentences(article, entity):
    sentences = [sent.text for sent in article.sents]
    entities = [ent.text for ent in article.ents if ent.label_ == entity]
    sent_w_ent = []
    for s in sentences:
        if any(ent in s for ent in entities):
            sent_w_ent.append(s)
    
    return sent_w_ent

In [26]:
len(filter_sentences(processed[0],"PERSON"))

249

In [27]:
people_found = []
for i in tqdm(range(len(processed))):
    people_found.extend(filter_sentences(processed[i],"PERSON"))

100%|██████████| 730/730 [00:56<00:00, 12.85it/s]


In [28]:
organisations_found = []
for i in tqdm(range(len(processed))):
    organisations_found.extend(filter_sentences(processed[i],"ORG"))

100%|██████████| 730/730 [00:43<00:00, 16.66it/s]


In [30]:
len(people_found)

349534

In [60]:
ceo_df = pd.DataFrame({'text_persons':people_found})

In [32]:
company_df = pd.DataFrame({'text_organisations':organisations_found})

In [33]:
ceo_df

,text_persons
0,From the report: With the House prepared to v...
1,Here's the banner leading Drudge Report right ...
2,South Korea -- whose heavy reliance on global ...
3,From the report: The HSBC South Korea Purchasi...
4,Markit UPDATE: As the Fiscal Cliff bill gets c...
...,...
349529,Drillers are already slashing billions of doll...
349530,One administration official said the question ...
349531,"Even so, with no changes to the core U.S. law ..."
349532,Whether to allow crude oil exports directly be...


In [55]:
def match(sentence, label_lst):
    '''
    Iterates through list of substrings to see if any are in a sentence.
    Used for labeling the data as positive and negative samples generally. 
    '''
    if any(lbl in sentence for lbl in label_lst):
        return 1
    else:
        return 0

In [56]:
def company_word(sentence):
    if (bool(re.search(r'(Advisors|Partner|LP|Associate|Co|Group|LTD|AirLL|Management|Capital)',sentence))):
        return 1
    else: 
        return 0

In [57]:
def num_capitals(phrase):
    cap_count = 0
    for letter in phrase:
        if letter.isupper():
            cap_count = cap_count + 1
    return(cap_count)

In [61]:
def num_words(phrase):
    return len(phrase.split())

In [62]:
%time ceo_df['num_capitals'] = ceo_df['text_persons'].apply(lambda x: (num_capitals(x)))

CPU times: user 3.17 s, sys: 12 ms, total: 3.18 s
Wall time: 3.19 s


In [63]:
ceo_df

,text_persons,num_capitals
0,From the report: With the House prepared to v...,19
1,Here's the banner leading Drudge Report right ...,6
2,South Korea -- whose heavy reliance on global ...,5
3,From the report: The HSBC South Korea Purchasi...,17
4,Markit UPDATE: As the Fiscal Cliff bill gets c...,11
...,...,...
349529,Drillers are already slashing billions of doll...,1
349530,One administration official said the question ...,1
349531,"Even so, with no changes to the core U.S. law ...",3
349532,Whether to allow crude oil exports directly be...,12


In [64]:
%time ceo_df['ceo_label'] = ceo_df['text_persons'].apply(lambda x: (match(x,train_ceo)))

CPU times: user 1min 5s, sys: 218 ms, total: 1min 5s
Wall time: 1min 6s


In [65]:
ceo_df

,text_persons,num_capitals,ceo_label
0,From the report: With the House prepared to v...,19,1
1,Here's the banner leading Drudge Report right ...,6,0
2,South Korea -- whose heavy reliance on global ...,5,0
3,From the report: The HSBC South Korea Purchasi...,17,0
4,Markit UPDATE: As the Fiscal Cliff bill gets c...,11,0
...,...,...,...
349529,Drillers are already slashing billions of doll...,1,0
349530,One administration official said the question ...,1,0
349531,"Even so, with no changes to the core U.S. law ...",3,0
349532,Whether to allow crude oil exports directly be...,12,1


In [66]:
%time ceo_df['num_words'] = ceo_df['text_persons'].apply(lambda x: (num_words(x)))

CPU times: user 602 ms, sys: 4.62 ms, total: 607 ms
Wall time: 607 ms


In [67]:
ceo_df

,text_persons,num_capitals,ceo_label,num_words
0,From the report: With the House prepared to v...,19,1,40
1,Here's the banner leading Drudge Report right ...,6,0,16
2,South Korea -- whose heavy reliance on global ...,5,0,29
3,From the report: The HSBC South Korea Purchasi...,17,0,31
4,Markit UPDATE: As the Fiscal Cliff bill gets c...,11,0,31
...,...,...,...,...
349529,Drillers are already slashing billions of doll...,1,0,34
349530,One administration official said the question ...,1,0,28
349531,"Even so, with no changes to the core U.S. law ...",3,0,30
349532,Whether to allow crude oil exports directly be...,12,1,62


In [43]:
%time company_df['num_words'] = company_df['text_organisations'].apply(lambda x: (num_words(x)))

CPU times: user 1.07 s, sys: 7.08 ms, total: 1.08 s
Wall time: 1.08 s


In [83]:
company_df

,text_organisations,num_words,num_capitals,company_nearby,company_label
0,Earlier today we had a strong South Korean PMI...,10,6,0,0
1,The latest?,2,1,0,0
2,It just saw a rise in December PMI from 47.4 t...,12,5,0,0
3,From the report: With the House prepared to v...,40,19,1,0
4,Here's the banner leading Drudge Report right ...,16,6,0,0
...,...,...,...,...,...
671738,Drillers are already slashing billions of doll...,34,1,0,0
671739,One administration official said the question ...,28,1,0,0
671740,"Even so, with no changes to the core U.S. law ...",30,3,0,0
671741,Whether to allow crude oil exports directly be...,62,12,1,0


In [45]:
%time company_df['num_capitals'] = company_df['text_organisations'].apply(lambda x: (num_capitals(x)))

CPU times: user 5.53 s, sys: 14.1 ms, total: 5.54 s
Wall time: 5.55 s


In [46]:
company_df

,text_organisations,num_words,num_capitals
0,Earlier today we had a strong South Korean PMI...,10,6
1,The latest?,2,1
2,It just saw a rise in December PMI from 47.4 t...,12,5
3,From the report: With the House prepared to v...,40,19
4,Here's the banner leading Drudge Report right ...,16,6
...,...,...,...
671738,Drillers are already slashing billions of doll...,34,1
671739,One administration official said the question ...,28,1
671740,"Even so, with no changes to the core U.S. law ...",30,3
671741,Whether to allow crude oil exports directly be...,62,12


In [47]:
%time company_df['company_nearby'] = company_df['text_organisations'].apply(lambda x: company_word(x))

CPU times: user 1.15 s, sys: 5.81 ms, total: 1.16 s
Wall time: 1.16 s


In [48]:
company_df

,text_organisations,num_words,num_capitals,company_nearby
0,Earlier today we had a strong South Korean PMI...,10,6,0
1,The latest?,2,1,0
2,It just saw a rise in December PMI from 47.4 t...,12,5,0
3,From the report: With the House prepared to v...,40,19,1
4,Here's the banner leading Drudge Report right ...,16,6,0
...,...,...,...,...
671738,Drillers are already slashing billions of doll...,34,1,0
671739,One administration official said the question ...,28,1,0
671740,"Even so, with no changes to the core U.S. law ...",30,3,0
671741,Whether to allow crude oil exports directly be...,62,12,1


In [49]:
%time company_df['company_label'] = company_df['text_organisations'].apply(lambda x: (match(x,train_corp)))

CPU times: user 5min 19s, sys: 990 ms, total: 5min 20s
Wall time: 5min 22s


In [50]:
company_df

,text_organisations,num_words,num_capitals,company_nearby,company_label
0,Earlier today we had a strong South Korean PMI...,10,6,0,0
1,The latest?,2,1,0,0
2,It just saw a rise in December PMI from 47.4 t...,12,5,0,0
3,From the report: With the House prepared to v...,40,19,1,0
4,Here's the banner leading Drudge Report right ...,16,6,0,0
...,...,...,...,...,...
671738,Drillers are already slashing billions of doll...,34,1,0,0
671739,One administration official said the question ...,28,1,0,0
671740,"Even so, with no changes to the core U.S. law ...",30,3,0,0
671741,Whether to allow crude oil exports directly be...,62,12,1,0


In [51]:
company_df[['company_nearby','company_label','num_capitals','num_words']].describe()
#the results from this indicate that roughly 7% of the sentences contain matches
#given almost 350k sentences containing persons, 7% would be 24,500 sentences containing matched ceos
#this seems reasonable

,company_nearby,company_label,num_capitals,num_words
count,671743.000000,671743.000000,671743.000000,671743.000000
mean,0.072298,0.197063,4.272957,21.044947
std,0.258982,0.397781,7.264835,12.935444
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,12.000000
50%,0.000000,0.000000,2.000000,19.000000
75%,0.000000,0.000000,5.000000,28.000000
max,1.000000,1.000000,1863.000000,1086.000000


### Machine Learning Classification - Company

In [68]:
from sklearn.model_selection import train_test_split
X = company_df[['company_nearby','num_capitals','num_words']]
y = company_df.company_label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [69]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [72]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.83      0.97      0.89    215769
           1       0.61      0.17      0.27     52929

    accuracy                           0.82    268698
   macro avg       0.72      0.57      0.58    268698
weighted avg       0.79      0.82      0.77    268698



In [74]:
y_test.mean() #beat the null(?)

0.19698323024361922

In [76]:
classified_samples = X_test[y_test == predictions]

In [89]:
pd.merge(company_df, classified_samples, left_index=True, right_index=True)

,text_organisations,num_words_x,num_capitals_x,company_nearby_x,company_label,company_nearby_y,num_capitals_y,num_words_y
2,It just saw a rise in December PMI from 47.4 t...,12,5,0,0,0,5,12
4,Here's the banner leading Drudge Report right ...,16,6,0,0,0,6,16
8,Following six months of continuous decline new...,15,1,0,0,0,1,15
9,"However, the rate of growth was only slight, w...",21,1,0,0,0,1,21
11,New export orders fell during December for the...,12,2,0,0,0,2,12
...,...,...,...,...,...,...,...,...
671732,But several dozen other companies that also ra...,20,1,0,0,0,1,20
671733,The agency also released its first ever writte...,37,4,0,0,0,4,37
671735,"On Tuesday, the Brent/WTI spread narrowed by m...",16,6,0,0,0,6,16
671739,One administration official said the question ...,28,1,0,0,0,1,28


In [ ]:
%%time
final_companies_list = []
for i in company_df['text_organisations']:
    final_comp = matching_engine_3000(nlp(i),"ORG")
    final_companies_list.extend(final_comp)

In [ ]:
final_companies_list = pd.DataFrame(final_companies_list)
final_companies_list.to_csv('final_companies_list.csv',index=False)

### Machine Learning Classification - CEOs

In [ ]:
from sklearn.model_selection import train_test_split
X = company_df[['company_nearby','num_capitals','num_words']]
y = company_df.company_label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)
predictions = logmodel.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))

In [ ]:
y_test.mean() #beat the null(?)

In [ ]:
classified_samples = X_test[y_test == predictions]

In [ ]:
pd.merge(ceo_df, classified_samples, left_index=True, right_index=True)

In [ ]:
%%time
final_ceos_list = []
for i in company_df['text_persons']:
    final_ceos = matching_engine_3000(nlp(i),"PERSON")
    final_ceos_list.extend(final_ceos)
    

In [ ]:
final_ceos_list = pd.DataFrame(final_ceos_list)
final_ceos_list.to_csv('final_ceos_list.csv',index=False)

### End of Task. 
#### Beyond this point is a skeleton yard of code which includes:
1. Machine Learning Pipeline (turns out Logistic Regression has the highest classification rates)
2. Recurrent Neural Nets are computationally expensive and difficult to implement.
3. Old pickling.

------------

In [ ]:
# import pandas
# import matplotlib.pyplot as plt
# from sklearn import model_selection
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# # load dataset
# X = company_df[['company_nearby','num_capitals','num_words']]
# Y = company_df.company_label

# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
# # prepare configuration for cross validation test harness
# seed = 7
# # prepare models
# models = []
# models.append(('LR', LogisticRegression()))
# models.append(('LDA', LinearDiscriminantAnalysis()))
# models.append(('KNN', KNeighborsClassifier()))
# models.append(('CART', DecisionTreeClassifier()))
# models.append(('NB', GaussianNB()))
# models.append(('SVM', SVC()))
# # evaluate each model in turn
# results = []
# names = []
# scoring = 'accuracy'
# for name, model in models:
# 	kfold = model_selection.KFold(n_splits=10, random_state=seed)
# 	cv_results = model_selection.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
# 	results.append(cv_results)
# 	names.append(name)
# 	msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
# 	print(msg)
# # boxplot algorithm comparison
# fig = plt.figure()
# fig.suptitle('Algorithm Comparison')
# ax = fig.add_subplot(111)
# plt.boxplot(results)
# ax.set_xticklabels(names)
# plt.show()

In [ ]:
# def posn_finder(article, labels, entity_name):
#     result = []
#     article_labels = [x for x in labels if x in article]
#     if len(article_labels) == 0:
#         # if empty return empty list
#         return article_labels
#     else: 
#         for label in article_labels:
#             entities = [(m.start(), m.end(), entity_name) for m in re.finditer(label, article)]
#             result.extend(entities)
    
#     return result

In [ ]:
# %%time
# ceos_matched = []
# for doc in content:
#     ceo_position = posn_finder(doc, train_ceo, "CEO")
#     ceos_matched.append(ceo_position)

In [ ]:
# %%time
# companies_matched = []
# for doc in content:
#     comp_positions = posn_finder(doc, train_corp, "Company")
#     companies_matched.append(comp_positions)

In [ ]:
# combined_data = []
# for i in range(len(content)):
#     combined_data.append(ceos_matched[i] + companies_matched[i])

In [ ]:
# space_station = [(content[q],{'entities': combined_data[q]}) for q in range(len(content))]
# 

In [ ]:
# with open('space_station.pkl','wb') as pf:
#     pickle.dump(space_station,pf,protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# with open('space_station.pkl','rb') as pf:
#     space_station = pickle.load(pf)

In [ ]:
# space_station[1]

### Build a Redundent Recurrent Neural Net

In [ ]:
# from __future__ import unicode_literals, print_function

# import plac #  wrapper over argparse
# import random
# from pathlib import Path
# import spacy
# from tqdm import tqdm # loading bar

In [ ]:
# train_space = random.sample(space_station,10)
# len(train_space)

In [ ]:
# # Define our variables
# model = None
# output_dir=Path("/Users/saifbhatti/Desktop/Northwestern/sy1920/w20/iems308/iems308-saifbhatti/homew3/")
# n_iter=10

In [ ]:
# nlp = spacy.blank("en")
# ner = nlp.create_pipe("ner")
# ner.add_label('CEO')
# ner.add_label('Company')
# nlp.add_pipe(ner, last=True)
# ner = nlp.get_pipe("ner")

In [ ]:
# def create_blank_nlp(train_data):
#     nlp = spacy.blank("en")
#     ner = nlp.create_pipe("ner")
#     ner.add_label('CEO')
#     ner.add_label('Company')
#     nlp.add_pipe(ner, last=True)
#     ner = nlp.get_pipe("ner")
#     for _, annotations in train_data:
#         for ent in annotations.get("entities"):
#             ner.add_label(ent[2])
#     return nlp

In [ ]:
# from spacy.util import minibatch, compounding

In [ ]:
# nlp = create_blank_nlp(train_space)
# optimizer = nlp.begin_training()
# for i in tqdm(range(3)):
#     losses = {}
#     batches = minibatch(train_space, size=compounding(4.0, 32.0, 1.001))
#     for batch in batches:
#         texts, annotations = zip(*batch)
#         nlp.update(
#             texts,  # batch of texts
#             annotations,  # batch of annotations
#             drop=0.1,  # dropout - make it harder to memorise data
#             losses=losses,
#         )
#     print(losses)

In [ ]:
# # add labels
# for _, annotations in train_space:
#     for ent in annotations.get('entities'):
#         ner.add_label(ent[2])

#     # get names of other pipes to disable them during training
# other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
# with nlp.disable_pipes(*other_pipes):  # only train NER
#     optimizer = nlp.begin_training()
#     for itn in range(n_iter):
#         random.shuffle(train_space)
#         losses = {}
        
#         batches = minibatch(train_space, size=compounding(4.0, 32.0, 1.001))
#         for batch in tqdm(batches):
#             text,annotations=zip(*batch)
#             nlp.update(
#                 [text],  # batch of texts
#                 [annotations],  # batch of annotations
#                 drop=0.5,  # dropout - make it harder to memorise data
#                 sgd=optimizer,  # callable to update weights
#                 losses=losses)
#         print(losses)





In [ ]:
# from spacy.util import minibatch, compounding
# # get names of other pipes to disable them during training
# pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
# other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
# with nlp.disable_pipes(*other_pipes):  # only train NER
#     # reset and initialize the weights randomly – but only if we're
#     # training a new model
#     if model is None:
#         nlp.begin_training()
#     for itn in range(n_iter):
#         random.shuffle(train_space)
#         losses = {}
#         # batch up the examples using spaCy's minibatch
#         batches = minibatch(train_space, size=compounding(4.0, 32.0, 1.001))
#         for batch in tqdm(batches):
#             texts, annotations = zip(*batch)
#             nlp.update(
#                 texts,  # batch of texts
#                 annotations,  # batch of annotations
#                 drop=0.5,  # dropout - make it harder to memorise data
#                 losses=losses,
#             )
#         print("Losses", losses)

In [ ]:
# %%time 
# import nltk

# bi_corpra=[]
# bi_=nltk.bigrams(preprocessed_list)
# bi_ = list(bi_)
# for i in bi_:
#     bi_corpra.append(' '.join(i))

# %%time 
# tri_corpra=[]
# tri_=nltk.trigrams(preprocessed_list)
# tri_ = list(tri_)
# for i in tri_:
#     tri_corpra.append(' '.join(i))

### Remaining steps:

1. Import training labels and lemmatize.
2. Use `regex` to determine list of matches between training files and corpus.
    1. Create feature tables from matchable training data.
        1. Create `ceo`,`percentage`, `corporation` dataframe with training data.
3. Negative sampling
    1. Redetermine POS based on lemmatized tokens.
    2. Identify `person` and `organisation` types using `NER`.
    3. Create `ceo`,`percentage`, `corporation` negative samples list.
    4. Refine negative samples to remove possible positives (manual selection process).
    5. Add this to `ceo`,`percentage`, `corporation` dataframes.
4. Begin regression to classify `ceo`,`percentage`, `corporation`.
5. Loop through percentages using `regex` and construct dataframe.
    1. Add results to `percentage` dataframe as features.
6. Find 'nearby' content for `ceo` and `corporation`.
    1. Add results to respective dataframes as features.
7. Apply classification model to test data.
8. Extract `ceo`,`percentage`, `corporation`
    1. Eliminate rows that don't meet threshold likelihood.
    2. Extract matching values.
    3. Store to csv.